<a href="https://colab.research.google.com/github/beliaazzhr/TeamBased--Ensemble-Learning/blob/main/tugas3_seed_1301200048.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Isi nama dan NIM seluruh anggota kelompok di sini
1. Anyelir Belia Azzahra - 1301200048
2. Fakhri Maulana Falah - 1301202117
3. Gilang Satya Nugraha - 1301202319
4. Mirai Tsuciya   - 1301203555


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree


# Load Dataset
Inisialisasi variable, masukan NIM di sini untuk random state.

Gunakan NIM salah satu anggota kelompok.



In [ ]:
filepath = "./seeds_dataset.csv"

# Isi nim anda di sini
nim = 1301200048

Load dataset, cek konten data

Target kelas: **Category** 

In [ ]:
df = pd.read_csv(filepath, sep="\t")
df.sample(10, random_state=nim)

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len,Category
172,11.27,12.97,0.8419,5.088,2.763,4.309,5.000,3
163,12.55,13.57,0.8558,5.333,2.968,4.419,5.176,3
193,10.82,12.83,0.8256,5.180,2.630,4.853,5.089,3
179,12.21,13.47,0.8453,5.357,2.893,1.661,5.178,3
197,13.37,13.78,0.8849,5.320,3.128,4.670,5.091,3
155,11.19,13.05,0.8253,5.250,2.675,5.813,5.219,3
16,13.99,13.83,0.9183,5.119,3.383,5.234,4.781,1
44,15.11,14.54,0.8986,5.579,3.462,3.128,5.180,1
26,13.02,13.76,0.8641,5.395,3.026,3.373,4.825,1
157,12.13,13.73,0.8081,5.394,2.745,4.825,5.220,3


Training/test split **60/40**

In [ ]:
# X: semua kolom, Y: kolom terakhir (category)
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6, random_state=nim)

# We perform bootstrap on the train data
# Keep test data for final accuracy testing
len(x_train), len(x_test)

(126, 84)

# Random Forest
Build a Random Forest, consisting of 20 Decision Tree classifer, with the following steps:

1. Prepare Bootstrap data (choose randomly from the *training data*, use sampling with replacement)
2. Choose 2-4 random features/atributes for each tree
3. Build Decision Tree classifier using step (1) and (2)
4. Calculate OOB score within the tree only

**NOTE**
This whole exercise has been simplified.
- Tree can be created using a library, therefore you may not be able control the feature selection on node level
- Random features are only selected in the beginning of each tree.
- OOB score is only calculated within its own tree instead.
- Typically Random Forest don't use train/test split and can be validated using OOB error. We simplify this to make it easier to test.
- You are not allowed to use a Random Forest or Bagging library.


In [ ]:
np.random.seed(nim)
n_tree = 20

forest = []
bootstrap_columns = []
for i in range(n_tree):
    # 1. Bootstrapping
    # select random rows
    rows = np.random.choice(x_train.index, size=len(x_train), replace=True)
    x_bootstrap = x_train.loc[rows]
    y_bootstrap = y_train.loc[rows]

    # Out of Bag data
    non_selected_rows = np.setdiff1d(x_train.index, rows)
    x_oob = x_train.loc[non_selected_rows]
    y_oob = y_train.loc[non_selected_rows]

    # 2. Select random features (make sure to keep track the selected columns)
    # Check for any repeated combinations
    feature_taken = np.random.randint(2, 4)
    cols = np.random.choice(x_bootstrap.columns, size=feature_taken, replace=True)
    bootstrap_columns.append(cols)

    # filter based on the selected columns
    x_bootstrap = x_bootstrap.loc[:, cols]

    # 3. Build decision tree
    # Let it grow, no pruning
    tree_model = tree.DecisionTreeClassifier()
    tree_model.fit(x_bootstrap, y_bootstrap)

    # Add it to the forest
    forest.append(tree_model)

    # 4. Calculate oob score only within its own tree
    # ....
    oob_predict = tree_model.predict(x_oob.loc[:, cols])
    print(cols)
    oob_score   = tree_model.score(x_oob.loc[:, cols], y_oob)
    print(f"Tree {i}, {len(x_oob)} data, oob score: {oob_score}")



['Area' 'Kernel_len' 'Compactness']
Tree 0, 42 data, oob score: 0.7619047619047619
['Assymetry' 'Compactness' 'Area']
Tree 1, 36 data, oob score: 0.8611111111111112
['Perimeter' 'Area']
Tree 2, 50 data, oob score: 0.9
['Perimeter' 'Groove_len']
Tree 3, 46 data, oob score: 0.9565217391304348
['Kernel_width' 'Perimeter']
Tree 4, 45 data, oob score: 0.8444444444444444
['Kernel_len' 'Perimeter' 'Assymetry']
Tree 5, 49 data, oob score: 0.8163265306122449
['Kernel_len' 'Kernel_len' 'Assymetry']
Tree 6, 45 data, oob score: 0.8
['Kernel_len' 'Groove_len']
Tree 7, 49 data, oob score: 0.7959183673469388
['Kernel_len' 'Compactness' 'Perimeter']
Tree 8, 46 data, oob score: 0.8043478260869565
['Area' 'Kernel_width' 'Assymetry']
Tree 9, 44 data, oob score: 0.8636363636363636
['Area' 'Compactness']
Tree 10, 38 data, oob score: 0.8421052631578947
['Kernel_len' 'Compactness']
Tree 11, 41 data, oob score: 0.8780487804878049
['Assymetry' 'Compactness' 'Area']
Tree 12, 46 data, oob score: 0.78260869565217

# Prediction and Voting
Run prediction on x_test data for each tree

Also calculate accuracy per tree here

In [ ]:
all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test.loc[:, bootstrap_columns[i]]

    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...

    acc = tree_model.score(x_test_filtered, y_test)
    print(acc)

    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)

0.7380952380952381
0.75
0.7857142857142857
0.8928571428571429
0.8095238095238095
0.7857142857142857
0.7380952380952381
0.7857142857142857
0.7738095238095238
0.8690476190476191
0.7738095238095238
0.75
0.7976190476190477
0.7738095238095238
0.7976190476190477
0.8214285714285714
0.75
0.7976190476190477
0.8333333333333334
0.5595238095238095


### Perform voting on the predictions from the Random Forest

In [ ]:
# Insert voting code here
voted_predictions = np.array([np.bincount(pred).argmax() for pred in all_preds.T])
print(voted_predictions)

# Calculate accuracy of Test data on the Forest
acc = np.sum(voted_predictions == y_test) / len(y_test) *100
print(acc, "%")


[3 3 3 3 1 3 1 1 1 3 3 1 2 3 2 2 2 1 3 2 1 1 3 3 1 1 2 1 1 3 2 3 2 3 1 1 1
 2 1 2 3 1 3 3 2 2 1 2 3 2 2 1 3 2 1 1 2 2 1 2 2 1 1 3 2 3 2 1 1 1 2 1 1 2
 1 3 1 1 2 1 2 3 3 2]
85.71428571428571 %


After you're done with all this, please check the excel file and put the answers for every column there.